In [1]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import random as rd
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder

/Users/Optimus-Prime/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/Optimus-Prime/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
datadir1 = '/Users/Optimus-Prime/Desktop/homework/dset1/train'
datadir2 = '/Users/Optimus-Prime/Desktop/homework/dset2/train'

### 定义网络参数

In [3]:
learning_rate = 0.001
epoch = 30000             
batch_size = 300
display_step = 1

In [4]:
n_input = 224*224*3
n_classes = 65
dropout = 0.75

In [5]:
x = tf.placeholder(tf.float32,[None,128,128,3])
y = tf.placeholder(tf.int32,[None,n_classes])
keep_prob = tf.placeholder(tf.float32)

In [6]:
def Reading(datadir,nameinput):
    classes = os.listdir(datadir)
    classes.remove('.DS_Store')

    img_path_total = []
    img_label_total = []
    for name in classes:
        class_path = datadir + '/'+name + '/'
        index = int(name[5:])
        for img_name in os.listdir(class_path):
            img_path_total.append(class_path + img_name)  # 每张图片的地址
            img_label_total.append(index) #每张图片的标签
    img_total = [[img_label_total[i],img_path_total[i]] for i in range(len(img_label_total))]
    #print(img_total)
    rd.shuffle(img_total)
    #print(img_total)
    y_label, x_path = [img_total[i][0] for i in range(len(img_label_total))],[img_total[i][1] for i in range(len(img_label_total))]
    
    #for i in range(len(img_total)):
    x_train_path, x_test_path, y_train, y_test = train_test_split(x_path, y_label, test_size = 0.2)  

    writer = tf.python_io.TFRecordWriter(nameinput+'_train'".tfrecords")  # 要生成的文件
    for i in range(len(y_train)):
            img = Image.open(x_train_path[i])
            img = img.resize((224, 224))
            img_raw = img.tobytes()  # 将图片转化为二进制格式
            example = tf.train.Example(features=tf.train.Features(feature={
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[y_train[i]])),
                'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
            }))  # example对象对label和image数据进行封装
            writer.write(example.SerializeToString())  # 序列化为字符串
    writer.close()
    
    writer = tf.python_io.TFRecordWriter(nameinput+'_test'".tfrecords")  # 要生成的文件
    for i in range(len(y_test)):
            img = Image.open(x_test_path[i])
            img = img.resize((224, 224))
            img_raw = img.tobytes()  # 将图片转化为二进制格式
            example = tf.train.Example(features=tf.train.Features(feature={
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[y_test[i]])),
                'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
            }))  # example对象对label和image数据进行封装
            writer.write(example.SerializeToString())  # 序列化为字符串
    writer.close()

In [7]:
Reading(datadir1,'data1')
Reading(datadir2,'data2')

In [20]:
def read_and_decode(filename): # 读入dog_train.tfrecords
    filename_queue = tf.train.string_input_producer([filename],num_epochs=30)#生成一个queue队列
 
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)#返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })#将image数据和label取出来
 
    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [224, 224, 3])  #reshape为128*128的3通道图片
    img = tf.cast(img, tf.float32) * (1. / 255) - 0.5 #在流中抛出img张量
    label = tf.cast(features['label'], tf.int32) #在流中抛出label张量
    return img, label

In [21]:
image1_train,label1_train = read_and_decode('./data1_train.tfrecords')
image2_train,label2_train = read_and_decode('./data2_train.tfrecords')

In [11]:
'''
image1_train, label1_train = g_batches("./data1_train.tfrecords",30)
with tf.Session() as sess:
    init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    for i in range(10):
        label1_train_ = sess.run(label1_train)
        print(label1_train_)
    coord.request_stop()
    coord.join(threads)
sess.close()
'''

'\nimage1_train, label1_train = g_batches("./data1_train.tfrecords",30)\nwith tf.Session() as sess:\n    init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())\n    sess.run(init)\n    coord = tf.train.Coordinator()\n    threads = tf.train.start_queue_runners(sess=sess,coord=coord)\n    for i in range(10):\n        label1_train_ = sess.run(label1_train)\n        print(label1_train_)\n    coord.request_stop()\n    coord.join(threads)\nsess.close()\n'

In [22]:
def g_batches(filename,batchsize):
    img, label = read_and_decode(filename)
    min_after_dequeue = 100
    capacity = min_after_dequeue + 3*batchsize
    
    img_batch, label_batch = tf.train.shuffle_batch([img, label],
                                                   batch_size = batchsize,
                                                   capacity = capacity,
                                                   min_after_dequeue = min_after_dequeue)
    return img_batch, label_batch

### 卷积全连接层

In [13]:
#卷积操作
def conv2d(name, x, W, b,strides = 1):
    x = tf.nn.conv2d(x, W, strides = [1,strides,strides,1], padding = 'SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x,name = name)
#定义pooling
def maxpool2d(name, x, k=2):
    return tf.nn.max_pool(x,ksize = [1,k,k,1],strides=[1,k,k,1],padding = 'SAME',name = name)
#BN
def B_Norm(name, l_input , lsize = 4):
    return tf.nn.lrn(l_input, lsize, bias = 1.0, alpha = 0.001/9, beta = 0.75, name = name)

#网络参数
weights = {
    'wc1': tf.Variable(tf.random_normal([11,11,3,48])),#224*224*3 ---> 16*16*48 (pooling : /2 ; stride = 4)
    'wc2': tf.Variable(tf.random_normal([5,5,48,48])), #16*16*48 ---> 8*8*48
    'wc3': tf.Variable(tf.random_normal([3,3,48,48])), #8*8*48 ---> 4*4*48
    'wf1': tf.Variable(tf.random_normal([768,256])), #768*256 ---> 256*n_classes
  # 'wf2': tf.Variable(tf.random_normal([1024,1024])),
    'out': tf.Variable(tf.random_normal([256,n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([48])),
    'bc2': tf.Variable(tf.random_normal([48])),
    'bc3': tf.Variable(tf.random_normal([48])),
    'bf1': tf.Variable(tf.random_normal([256])),
   #'bf2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [14]:
def cnn_net(x,weights,biases,dropout):
    x = tf.reshape(x, shape = [-1,128,128,3])
    #第一层
    conv1 = conv2d('conv1', x, weights['wc1'], biases['bc1'],strides = 4)
    pool1 = maxpool2d('pool1', conv1, k = 2)
    B_Norm1 = B_Norm('BN1', pool1, lsize = 4)
    #第二层
    conv2 = conv2d('conv2',B_Norm1,weights['wc2'], biases['bc2'])
    pool2 = maxpool2d('pool2',conv2, k = 2)
    B_Norm2 = B_Norm('BN2', pool2, lsize = 4)
    #第三层
    conv3 = conv2d('conv3',B_Norm2, weights['wc3'], biases['bc3'])
    pool3 = maxpool2d('pool3',conv3, k = 2)
    B_Norm3 = B_Norm('BN3', pool3, lsize = 4)
    
    #全连接层1
    fc1 = tf.reshape(B_Norm3, [-1,weights['wf1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1,weights['wf1']),biases['bf1'])
    fc1 = tf.nn.relu(fc1)
    #dropout
    fc1 = tf.nn.dropout(fc1,dropout)
    
    
    #输出
    out = tf.add(tf.matmul(fc1,weights['out']),biases['out'])
    return out

'''    #全连接层2
    fc2 = tf.reshape(fc1, [-1,weights['wf2'].get_shape().as_list()[0]])
    fc2 = tf.add(tf.matmul(fc2,weights['wf2']),biases['bf2'])
    fc2 = tf.nn.relu(fc2)
    #dropout
    fc2 = tf.nn.dropout(fc2,dropout)'''

"    #全连接层2\n    fc2 = tf.reshape(fc1, [-1,weights['wf2'].get_shape().as_list()[0]])\n    fc2 = tf.add(tf.matmul(fc2,weights['wf2']),biases['bf2'])\n    fc2 = tf.nn.relu(fc2)\n    #dropout\n    fc2 = tf.nn.dropout(fc2,dropout)"

In [30]:
pred = cnn_net(x,weights,biases,keep_prob)
#loss func
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
#evaluate func
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [23]:
batch_x_1, batch_y_1 = g_batches('data1_train.tfrecords',batch_size)
batch_y_1 = tf.one_hot(batch_y_1,n_classes,1,0)

valid_x_1, valid_y_1 = g_batches('data1_test.tfrecords',batch_size)
valid_y_1 = tf.one_hot(valid_y_1,n_classes,1,0)


In [40]:
with tf.Session() as sess: #开始一个会话
    init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord = coord)

    step = 1
    print("Start Session:\n")
    while step*batch_size < epoch:
        #print("after g_batch\n")
        x_, y_ = sess.run([batch_x_1,batch_y_1])
        #print("after transform to array\n")
        step += 1

        sess.run(optimizer,feed_dict={x: x_, y: y_, keep_prob: dropout})
        if not step%5:
            print('Print training accuracy:')
            loss_,acc_ = sess.run([loss,accuracy], feed_dict = {x: x_, y: y_, keep_prob: dropout})
            print("epoch" + str(step*batch_size) + ", Minibatch Loss = " + \
                  "{:.6f}".format(loss_) + ", Training Accuracy = " + \
                  "{:.5f}".format(acc_) + "\n")
            print(np.argmax(y_,1)[1:20],"\n",np.argmax(sess.run(pred,feed_dict = {x : x_, y : y_, keep_prob : dropout})[1:20],1))
        step += 1
    print("Optimization Finished")
    coord.request_stop()
    coord.join(threads)
    sess.close()

Start Session:

Print training accuracy:
epoch3000, Minibatch Loss = 8798.440430, Training Accuracy = 0.00667

[24 62 26 64  3 42 42 26 53 10 34 31 56 19 35 60 14 18 55] 
 [58 30 17 25 58 56 42 58 27 30  6 29 27 30 42 14 53 42 42]
Print training accuracy:
epoch6000, Minibatch Loss = 7321.375977, Training Accuracy = 0.01000

[40 42  1  4 56  0 22  4  8 48 53  9 55 10  1 28 58 34 32] 
 [30 60 30 42 60 61 29 30 45 27 28  8 43 42 30 52 58 42 25]
Print training accuracy:
epoch9000, Minibatch Loss = 6306.354004, Training Accuracy = 0.02333

[54  1 63 11  4 40 30 21 57 12 14 11 17 50 25 25 58 47 24] 
 [45 42 36 60 28 27 45 42 36 27 27  8 13 58 28 28 23 47 27]
Print training accuracy:
epoch12000, Minibatch Loss = 4975.745117, Training Accuracy = 0.02000

[56 20 45 52 18 28 50 42 56 52 18 37 23 50 43 21 12 11 23] 
 [43 58  1 53 29 23 32 25 60 40 30 11 42 39 42 58 39 36 27]
Print training accuracy:
epoch15000, Minibatch Loss = 4509.710449, Training Accuracy = 0.02000

[52 20 24 26  4 27  6 13 31

In [10]:
with tf.Session() as sess: #开始一个会话
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    coord=tf.train.Coordinator()
    threads= tf.train.start_queue_runners(coord=coord)
    cwd = '/Users/Optimus-Prime/Desktop/test_image/'
    label1_train = sess.run(label1_train)
    yonehot1 = sess.run(tf.one_hot(label1_train,65,1,0))
    print(label1_train)
    print(yonehot1)
    '''for i in range(1000):
        example, l = sess.run([image1,label1])#在会话中取出image和label
        
        img=Image.fromarray(example, 'RGB')#这里Image是之前提到的
        img.save(cwd+str(i)+'_''Label_'+str(l)+'.jpg')#存下图片
        print(sess.run(image1),'end')'''
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


TypeError: Fetch argument 52 has invalid type <class 'numpy.int32'>, must be a string or Tensor. (Can not convert a int32 into a Tensor or Operation.)